In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn.functional as F
os.chdir("../")
from src.utilis.get_datasets import getDataTF, getDataHF
from src.utilis.models import decoding_pred
from src.data.processing_data import Format
from src.models.pipelines import Pipeline
from src.models.encoders import BERTencoder

2023-03-11 11:35:57.671111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 11:35:58.422948: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 11:35:58.423016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 11:35:58.423023: W 

In [13]:
_dataset = "meld_e"

In [14]:
df = pd.read_csv(f"./inputs_data/data_{_dataset}_train.csv", encoding="utf-8",  sep="|")
df.head()

In [16]:
# frequency of each dialog acts 
df.value_counts("Label")

Label
4    4710
3    1743
6    1205
0    1109
5     683
1     271
2     268
dtype: int64

In [17]:
# number of distinct label i the dataset
len(df["Label"].unique())

7

In [18]:
# number of distinct dialogue i the dataset
len(df["Dialogue_ID"].unique())

1038

In [19]:
# average number of utterance per dialogue
df[["Dialogue_ID", "Label"]].groupby("Dialogue_ID").count().Label.mean()

9.623314065510597

In [21]:
dimDialogAct, Contexts, Labels = Format(_dataset, 5, "stacked").get_contexts_labels()

In [22]:
len(Contexts[0])

806

In [23]:
Labels[0].shape

TensorShape([806, 35])

## Testing Pipeline working

In [25]:
Pipeline("meld_e", "stacked", 5, 'bert-base-uncased', "MLP", 2, 0.2).summary_exec()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predicti

Epoch 1/500
537/537 [==============================] - 3s 4ms/step - loss: 0.3632 - val_loss: 0.3604
Epoch 2/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3439 - val_loss: 0.3504
Epoch 3/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3374 - val_loss: 0.3494
Epoch 4/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3325 - val_loss: 0.3503
Epoch 5/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3265 - val_loss: 0.3433
Epoch 6/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3236 - val_loss: 0.3527
Epoch 7/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3188 - val_loss: 0.3441
Epoch 8/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3170 - val_loss: 0.3395
Epoch 9/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3135 - val_loss: 0.3444
Epoch 10/500
537/537 [==============================] - 2s 4ms/step - loss: 0.3115 - val_lo

,dataset_name,encoder_model,decoder_model,performance
0,meld_e,bert-base-uncased,MLP,0.486883
